In [1]:
123

123

Part 1: SparkSession 생성

In [3]:
# SparkSession 생성
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Day11-SparkBasics")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .getOrCreate()
)

print("SparkSession 생성 완료!")
print(f"  버전: {spark.version}")
print(f"  Master: {spark.sparkContext.master}")
print(f"  총 코어: {spark.sparkContext.defaultParallelism}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 05:35:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession 생성 완료!
  버전: 3.5.8
  Master: spark://spark-master:7077
  총 코어: 2


Part 2: 테스트 데이터 생성

In [4]:
# 테스트 데이터 생성
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

np.random.seed(42)


def generate_api_events(num_records):
    """API 이벤트 데이터 생성"""
    endpoints = ["/api/products", "/api/users", "/api/orders", "/api/payments", "/api/search"]
    methods = ["GET", "POST", "PUT", "DELETE"]
    status_codes = [200, 200, 200, 200, 201, 400, 404, 500]
    base_time = datetime(2024, 1, 1, 0, 0, 0)

    return pd.DataFrame({
        "request_id": [f"REQ_{i:08d}" for i in range(num_records)],
        "user_id": [f"U{np.random.randint(1, 1001):04d}" for _ in range(num_records)],
        "endpoint": np.random.choice(endpoints, num_records),
        "method": np.random.choice(methods, num_records),
        "status_code": np.random.choice(status_codes, num_records),
        "response_time_ms": np.random.randint(10, 500, num_records),
        "timestamp": [base_time + timedelta(seconds=i * 0.1) for i in range(num_records)],
    })


# 10만 건 데이터 생성
filepath = "/data/api_events_100k.csv"
if not os.path.exists(filepath):
    df = generate_api_events(100_000)
    df.to_csv(filepath, index=False)
    print(f"{filepath} 생성 완료 (100,000건)")
else:
    print(f"{filepath} 이미 존재")

/data/api_events_100k.csv 이미 존재


In [5]:
# 데이터 로드
df = spark.read.csv("/data/api_events_100k.csv", header=True, inferSchema=True)

print(f"데이터: {df.count():,}건")
print("\n스키마:")
df.printSchema()

26/01/19 05:35:23 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


데이터: 100,000건

스키마:
root
 |-- request_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- endpoint: string (nullable = true)
 |-- method: string (nullable = true)
 |-- status_code: integer (nullable = true)
 |-- response_time_ms: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
# 데이터 미리보기
df.show(5)

+------------+-------+-------------+------+-----------+----------------+--------------------+
|  request_id|user_id|     endpoint|method|status_code|response_time_ms|           timestamp|
+------------+-------+-------------+------+-----------+----------------+--------------------+
|REQ_00000000|  U0103|   /api/users|  POST|        200|             289| 2024-01-01 00:00:00|
|REQ_00000001|  U0436|/api/payments|   PUT|        400|             337|2024-01-01 00:00:...|
|REQ_00000002|  U0861|/api/products|  POST|        201|             427|2024-01-01 00:00:...|
|REQ_00000003|  U0271|  /api/orders|DELETE|        400|             409|2024-01-01 00:00:...|
|REQ_00000004|  U0107|/api/products|DELETE|        404|             392|2024-01-01 00:00:...|
+------------+-------+-------------+------+-----------+----------------+--------------------+
only showing top 5 rows



1. 컬럼 선택

In [7]:
# Pandas: df[["endpoint", "status_code", "response_time_ms"]]
# Spark:
selected = df.select("endpoint", "status_code", "response_time_ms")
selected.show(5)

+-------------+-----------+----------------+
|     endpoint|status_code|response_time_ms|
+-------------+-----------+----------------+
|   /api/users|        200|             289|
|/api/payments|        400|             337|
|/api/products|        201|             427|
|  /api/orders|        400|             409|
|/api/products|        404|             392|
+-------------+-----------+----------------+
only showing top 5 rows



2. 필터링

In [9]:
from pyspark.sql.functions import col

# Pandas: df[df["status_code"] >= 400]
# Spark:
errors = df.filter(col("status_code") >= 400)
print(f"에러 건수: {errors.count():,}건")
errors.show(5)

에러 건수: 37,665건
+------------+-------+-------------+------+-----------+----------------+--------------------+
|  request_id|user_id|     endpoint|method|status_code|response_time_ms|           timestamp|
+------------+-------+-------------+------+-----------+----------------+--------------------+
|REQ_00000001|  U0436|/api/payments|   PUT|        400|             337|2024-01-01 00:00:...|
|REQ_00000003|  U0271|  /api/orders|DELETE|        400|             409|2024-01-01 00:00:...|
|REQ_00000004|  U0107|/api/products|DELETE|        404|             392|2024-01-01 00:00:...|
|REQ_00000009|  U0122|  /api/search|   GET|        400|             249|2024-01-01 00:00:...|
|REQ_00000012|  U0331|/api/payments|   GET|        400|             300|2024-01-01 00:00:...|
+------------+-------+-------------+------+-----------+----------------+--------------------+
only showing top 5 rows



3. 새 컬럼 추가

In [10]:
from pyspark.sql.functions import when

# Pandas: df["is_error"] = df["status_code"] >= 400
# Spark:
df_with_flag = df.withColumn(
    "is_error",
    when(col("status_code") >= 400, True).otherwise(False)
)
df_with_flag.select("request_id", "status_code", "is_error").show(5)

+------------+-----------+--------+
|  request_id|status_code|is_error|
+------------+-----------+--------+
|REQ_00000000|        200|   false|
|REQ_00000001|        400|    true|
|REQ_00000002|        201|   false|
|REQ_00000003|        400|    true|
|REQ_00000004|        404|    true|
+------------+-----------+--------+
only showing top 5 rows



4. 그룹별 집계

In [11]:
from pyspark.sql.functions import count, avg, max as spark_max

# Pandas: df.groupby("endpoint").agg({"request_id": "count", "response_time_ms": "mean"})
# Spark:
endpoint_stats = df.groupBy("endpoint").agg(
    count("request_id").alias("요청수"),
    avg("response_time_ms").alias("평균응답시간"),
    spark_max("response_time_ms").alias("최대응답시간"),
)
endpoint_stats.show()

+-------------+------+------------------+------------+
|     endpoint|요청수|      평균응답시간|최대응답시간|
+-------------+------+------------------+------------+
|/api/products| 19982|254.71939745771195|         499|
|  /api/search| 20137|253.70283557630233|         499|
|  /api/orders| 19809|  254.466202231309|         499|
|   /api/users| 20119|254.30170485610617|         499|
|/api/payments| 19953|255.65869794015939|         499|
+-------------+------+------------------+------------+



5. 정렬

In [12]:
# Pandas: df.sort_values("response_time_ms", ascending=False)
# Spark:
sorted_df = df.orderBy(col("response_time_ms").desc())
sorted_df.select("request_id", "endpoint", "response_time_ms").show(5)

+------------+-------------+----------------+
|  request_id|     endpoint|response_time_ms|
+------------+-------------+----------------+
|REQ_00084401|   /api/users|             499|
|REQ_00001991|  /api/search|             499|
|REQ_00082961|/api/products|             499|
|REQ_00003287|/api/payments|             499|
|REQ_00082048|   /api/users|             499|
+------------+-------------+----------------+
only showing top 5 rows



SQL 쿼리 사용

In [17]:
# Spark의 강점: SQL로 데이터 처리 가능
df.createOrReplaceTempView("api_events")

result = spark.sql("""
    SELECT
        endpoint,
        COUNT(*) as `요청수`,
        ROUND(AVG(response_time_ms), 2) as `평균응답시간`,
        SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END) as `에러수`
    FROM api_events
    GROUP BY endpoint
    ORDER BY `요청수` DESC
""")
result.show()

+-------------+------+------------+------+
|     endpoint|요청수|평균응답시간|에러수|
+-------------+------+------------+------+
|  /api/search| 20137|       253.7|  7591|
|   /api/users| 20119|       254.3|  7567|
|/api/products| 19982|      254.72|  7539|
|/api/payments| 19953|      255.66|  7452|
|  /api/orders| 19809|      254.47|  7516|
+-------------+------+------------+------+



Part 4: Lazy Evaluation 실습

In [18]:
# Transformation은 실행되지 않음 (계획만 세움)
print("Transformation 정의 중... (아직 실행 안 됨)")

step1 = df.filter(col("status_code") >= 400)
step2 = step1.groupBy("endpoint").count()
step3 = step2.orderBy(col("count").desc())

print("여기까지 아무것도 실행되지 않았음!")
print("Action을 호출해야 실행됨")

Transformation 정의 중... (아직 실행 안 됨)
여기까지 아무것도 실행되지 않았음!
Action을 호출해야 실행됨


In [20]:
# Action 호출 → 모든 Transformation이 한번에 실행
print("\nAction 호출 (collect):")
result = step3.collect()
result


Action 호출 (collect):


[Row(endpoint='/api/search', count=7591),
 Row(endpoint='/api/users', count=7567),
 Row(endpoint='/api/products', count=7539),
 Row(endpoint='/api/orders', count=7516),
 Row(endpoint='/api/payments', count=7452)]

실행 계획 확인

In [21]:
# 실행 계획 보기
print("=== 실행 계획 ===")
step3.explain()

=== 실행 계획 ===
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(3) Sort [count#335L DESC NULLS LAST], true, 0
   +- AQEShuffleRead coalesced
      +- ShuffleQueryStage 1
         +- Exchange rangepartitioning(count#335L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=343]
            +- *(2) HashAggregate(keys=[endpoint#19], functions=[count(1)])
               +- AQEShuffleRead coalesced
                  +- ShuffleQueryStage 0
                     +- Exchange hashpartitioning(endpoint#19, 200), ENSURE_REQUIREMENTS, [plan_id=312]
                        +- *(1) HashAggregate(keys=[endpoint#19], functions=[partial_count(1)])
                           +- *(1) Project [endpoint#19]
                              +- *(1) Filter (isnotnull(status_code#21) AND (status_code#21 >= 400))
                                 +- FileScan csv [endpoint#19,status_code#21] Batched: false, DataFilters: [isnotnull(status_code#21), (status_code#21 >= 400)], Format: CSV, 

Part 5: 복합 분석 예제

In [ ]:
from pyspark.sql.functions import sum as spark_sum

# 엔드포인트별 에러율 분석
error_analysis = (
    #엔드포인트별로 요청이 몇 개야?
    df.groupBy("endpoint")
    #agg()는 “여러 개 집계함수를 한 번에” 넣는 상자
    .agg(
        count("*").alias("총요청"),
        #그중에 에러(400 이상)는 몇 개야?
        spark_sum(when(col("status_code") >= 400, 1).otherwise(0)).alias("에러수"),
       #평균 응답 시간은?
        avg("response_time_ms").alias("평균응답시간"),
    )
    #새 컬럼 만들기
    .withColumn("에러율", col("에러수") / col("총요청") * 100)
    .orderBy(col("에러율").desc())
)

print("엔드포인트별 에러율 분석:")
error_analysis.show()

엔드포인트별 에러율 분석:
+-------------+------+------+------------------+------------------+
|     endpoint|총요청|에러수|      평균응답시간|            에러율|
+-------------+------+------+------------------+------------------+
|  /api/orders| 19809|  7516|  254.466202231309| 37.94234943712454|
|/api/products| 19982|  7539|254.71939745771195| 37.72895606045441|
|  /api/search| 20137|  7591|253.70283557630233|37.696777077022396|
|   /api/users| 20119|  7567|254.30170485610617| 37.61121328097818|
|/api/payments| 19953|  7452|255.65869794015939| 37.34776725304466|
+-------------+------+------+------------------+------------------+



In [23]:
# 응답시간 구간별 분포
from pyspark.sql.functions import when

response_dist = (
    df.withColumn(
        "응답시간구간",
        when(col("response_time_ms") < 100, "빠름 (<100ms)")
        .when(col("response_time_ms") < 300, "보통 (100-300ms)")
        .otherwise("느림 (>300ms)")
    )
    .groupBy("응답시간구간")
    .count()
    .orderBy("count")
)

print("응답시간 구간별 분포:")
response_dist.show()

응답시간 구간별 분포:
+----------------+-----+
|    응답시간구간|count|
+----------------+-----+
|   빠름 (<100ms)|18287|
|   느림 (>300ms)|40726|
|보통 (100-300ms)|40987|
+----------------+-----+



SparkSession 생성

In [2]:
# -----------------------------------------------------------------------------
# SparkSession 생성: 기본 형태
# -----------------------------------------------------------------------------
from pyspark.sql import SparkSession
# SparkSession.builder: SparkSession을 만들기 위한 빌더 객체 반환
# .appName("이름"): Spark UI에 표시될 애플리케이션 이름 설정
# .getOrCreate(): 새 세션 생성, 이미 있으면 기존 세션 반환
spark = SparkSession.builder \
    .appName("PySpark-API-Basics") \
    .getOrCreate()

# 생성된 세션 정보 확인
print("SparkSession 생성 완료!")
print(f"  버전: {spark.version}")                          # Spark 버전
print(f"  앱 이름: {spark.sparkContext.appName}")          # 앱 이름
print(f"  마스터: {spark.sparkContext.master}")            # 실행 모드 (local/cluster)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 07:10:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession 생성 완료!
  버전: 3.5.8
  앱 이름: PySpark-API-Basics
  마스터: local[*]


Builder 패턴으로 생성

In [ ]:
# -----------------------------------------------------------------------------
# SparkSession 생성: 설정 포함 (실무 패턴)
# -----------------------------------------------------------------------------

# 기존 세션 종료 (설정 변경을 위해)
# spark.stop()

# 새 세션 생성 (설정 포함)
spark = (
    SparkSession.builder
    # 애플리케이션 이름: Spark UI에서 식별용
    .appName("PySpark-API-Basics-Configured")

    # 실행 환경 설정
    # - "local[*]": 로컬 모드, 모든 코어 사용
    # - "spark://master:7077": 클러스터 모드
    .master("local[*]")

    # Executor 메모리: 각 워커가 사용할 메모리
    .config("spark.executor.memory", "2g")

    # 셔플 파티션 수: groupBy, join 등에서 사용
    # 기본값 200은 작은 데이터에 과도함 → 줄이면 성능 향상
    .config("spark.sql.shuffle.partitions", 50)

    # 세션 생성 또는 기존 세션 반환
    .getOrCreate()
)

print("설정된 SparkSession 생성 완료!")
print(f"  셔플 파티션: {spark.conf.get('spark.sql.shuffle.partitions')}")

설정된 SparkSession 생성 완료!
  셔플 파티션: 50


26/01/19 07:10:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


26/01/19 07:10:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Part 3: 데이터 읽기 (Read) 예시

In [27]:
spark.read\
    .format("csv")\
    .option("header", True)\
    .load("path/to/file")

# 또는 단축형
spark.read.csv("path", header=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/app/path/to/file.

In [6]:
# -----------------------------------------------------------------------------
# 테스트 데이터 생성 (Pandas로 먼저 생성 후 파일 저장)
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import os

# 재현 가능한 랜덤 시드 설정
np.random.seed(42)

# 샘플 데이터 생성: 직원 정보
sample_data = pd.DataFrame({
    "emp_id": [f"E{i:03d}" for i in range(1, 101)],          # E001 ~ E100
    "name": [f"Employee_{i}" for i in range(1, 101)],        # 이름
    "department": np.random.choice(                           # 부서 (랜덤)
        ["Engineering", "Sales", "Marketing", "HR", "Finance"],
        100
    ),
    "salary": np.random.randint(40000, 120000, 100),         # 연봉 (랜덤)
    "hire_date": pd.date_range("2020-01-01", periods=100, freq="7D"),  # 입사일
    "age": np.random.randint(25, 55, 100),                   # 나이 (랜덤)
    "is_manager": np.random.choice([True, False], 100, p=[0.2, 0.8]),  # 매니저 여부
})

# 결측치 일부 추가 (실무 데이터처럼)
sample_data.loc[5:10, "salary"] = None
sample_data.loc[15:18, "department"] = None

# 데이터 디렉토리 생성
os.makedirs("/tmp/spark_tutorial", exist_ok=True)
sample_data['hire_date'] = sample_data['hire_date'].astype("datetime64[us]")
# 다양한 포맷으로 저장
sample_data.to_csv("/tmp/spark_tutorial/employees.csv", index=False)
sample_data.to_parquet("/tmp/spark_tutorial/employees.parquet", index=False)
sample_data.to_json("/tmp/spark_tutorial/employees.json", orient="records", lines=True)

print("테스트 데이터 생성 완료!")
print(f"  행 수: {len(sample_data)}")
print(f"  컬럼: {list(sample_data.columns)}")

테스트 데이터 생성 완료!
  행 수: 100
  컬럼: ['emp_id', 'name', 'department', 'salary', 'hire_date', 'age', 'is_manager']


3-1. CSV 파일 읽기

In [7]:
# -----------------------------------------------------------------------------
# CSV 파일 읽기
# -----------------------------------------------------------------------------

# 방법 1: 기본 읽기 (헤더 있음, 스키마 자동 추론)
# - header=True: 첫 번째 줄을 컬럼명으로 사용
# - inferSchema=True: 데이터 타입 자동 추론 (숫자, 문자열 등)
#   주의: inferSchema는 데이터를 한 번 더 읽어서 느림 (대용량에서)
df_csv = spark.read.csv(
    "/tmp/spark_tutorial/employees.csv",  # 파일 경로
    header=True,                           # 첫 줄이 헤더인지
    inferSchema=True                       # 타입 자동 추론
)

# 스키마(구조) 확인
print("=== CSV 스키마 ===")
df_csv.printSchema()

=== CSV 스키마 ===
root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_manager: boolean (nullable = true)



3-2. Parquet 파일 읽기 (실무 표준)

In [8]:
# -----------------------------------------------------------------------------
# Parquet 파일 읽기
# -----------------------------------------------------------------------------

# Parquet은 스키마가 파일에 포함되어 있어 옵션이 거의 필요 없음
# inferSchema도 불필요 (자동으로 스키마 읽음)
df_parquet = spark.read.parquet("/tmp/spark_tutorial/employees.parquet")

print("=== Parquet 스키마 (자동 추론됨) ===")
df_parquet.printSchema()

=== Parquet 스키마 (자동 추론됨) ===
root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: timestamp_ntz (nullable = true)
 |-- age: long (nullable = true)
 |-- is_manager: boolean (nullable = true)



3-3. JSON 파일 읽기

In [9]:
# -----------------------------------------------------------------------------
# JSON 파일 읽기
# -----------------------------------------------------------------------------

# JSON Lines 형식 (한 줄에 하나의 JSON 객체)
# - 기본 JSON: [{"a":1}, {"a":2}]  → 배열
# - JSON Lines: {"a":1}\n{"a":2}   → 줄바꿈으로 구분
df_json = spark.read.json("/tmp/spark_tutorial/employees.json")

print("=== JSON 스키마 ===")
df_json.printSchema()
df_json.show(3)

=== JSON 스키마 ===
root
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- emp_id: string (nullable = true)
 |-- hire_date: long (nullable = true)
 |-- is_manager: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- salary: double (nullable = true)

+---+----------+------+----------+----------+----------+-------+
|age|department|emp_id| hire_date|is_manager|      name| salary|
+---+----------+------+----------+----------+----------+-------+
| 28|        HR|  E001|1577836800|     false|Employee_1|92251.0|
| 43|   Finance|  E002|1578441600|     false|Employee_2|62662.0|
| 50| Marketing|  E003|1579046400|     false|Employee_3|48392.0|
+---+----------+------+----------+----------+----------+-------+
only showing top 3 rows



3-4. 스키마 직접 지정 (권장)

In [10]:
# -----------------------------------------------------------------------------
# 스키마 직접 정의 후 읽기
# -----------------------------------------------------------------------------

# StructType: 전체 스키마 (테이블 구조)
# StructField(이름, 타입, nullable): 개별 컬럼 정의
#   - 이름: 컬럼명
#   - 타입: StringType(), IntegerType(), etc.
#   - nullable: NULL 허용 여부 (True/False)

from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType, BooleanType, DateType
)

# 스키마 정의
employee_schema = StructType([
    StructField("emp_id", StringType(), False),       # 필수 (nullable=False)
    StructField("name", StringType(), True),          # NULL 허용
    StructField("department", StringType(), True),    # NULL 허용
    StructField("salary", IntegerType(), True),       # 정수형
    StructField("hire_date", DateType(), True),       # 날짜형
    StructField("age", IntegerType(), True),          # 정수형
    StructField("is_manager", BooleanType(), True),   # 불린형
])

# 정의한 스키마로 CSV 읽기 (inferSchema 대신)
df_with_schema = (
    spark.read
    .option("header", "true")
    .schema(employee_schema)                          # 스키마 지정
    .csv("/tmp/spark_tutorial/employees.csv")
)

print("=== 스키마 직접 지정 결과 ===")
df_with_schema.printSchema()

=== 스키마 직접 지정 결과 ===
root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_manager: boolean (nullable = true)



Part 4: 데이터 쓰기 (Write)

In [11]:
# -----------------------------------------------------------------------------
# 데이터 쓰기: Parquet (실무 표준)
# -----------------------------------------------------------------------------

# mode("overwrite"): 기존 파일 덮어쓰기
# 주의: 실수로 데이터 날릴 수 있으니 경로 확인!
df_parquet.write \
    .mode("overwrite") \
    .parquet("/tmp/spark_tutorial/output/employees_output.parquet")

print("Parquet 저장 완료!")

Parquet 저장 완료!


In [12]:
# -----------------------------------------------------------------------------
# 데이터 쓰기: 파티셔닝 (대용량 데이터 필수)
# -----------------------------------------------------------------------------

# partitionBy(): 지정한 컬럼 값으로 폴더를 나눠서 저장
# 장점:
#   - 쿼리 시 필요한 파티션만 읽음 (Partition Pruning)
#   - department="Engineering" 조건 시 해당 폴더만 읽음
df_parquet.write \
    .mode("overwrite") \
    .partitionBy("department") \
    .parquet("/tmp/spark_tutorial/output/employees_partitioned")

print("파티셔닝 저장 완료!")
print("\n저장된 폴더 구조:")

# 폴더 구조 확인 (Bash 명령)
import subprocess
result = subprocess.run(
    ["find", "/tmp/spark_tutorial/output/employees_partitioned", "-type", "d"],
    capture_output=True, text=True
)
print(result.stdout)

파티셔닝 저장 완료!

저장된 폴더 구조:
/tmp/spark_tutorial/output/employees_partitioned
/tmp/spark_tutorial/output/employees_partitioned/department=__HIVE_DEFAULT_PARTITION__
/tmp/spark_tutorial/output/employees_partitioned/department=Engineering
/tmp/spark_tutorial/output/employees_partitioned/department=Sales
/tmp/spark_tutorial/output/employees_partitioned/department=HR
/tmp/spark_tutorial/output/employees_partitioned/department=Finance
/tmp/spark_tutorial/output/employees_partitioned/department=Marketing



In [13]:
# -----------------------------------------------------------------------------
# 데이터 쓰기: 파일 개수 조절
# -----------------------------------------------------------------------------

# 기본적으로 파티션 수만큼 파일이 생성됨
# coalesce(n): 파일 개수를 n개로 줄임 (셔플 없음, 감소만 가능)
# repartition(n): 파일 개수를 n개로 변경 (셔플 발생, 증가/감소 가능)

# 작은 파일 여러 개 → 큰 파일 하나로 합치기
df_parquet.coalesce(1) \
    .write \
    .mode("overwrite") \
    .parquet("/tmp/spark_tutorial/output/employees_single")

print("단일 파일로 저장 완료!")

단일 파일로 저장 완료!


In [14]:
# -----------------------------------------------------------------------------
# 데이터 쓰기: CSV (다른 시스템 연동용)
# -----------------------------------------------------------------------------

# CSV로 저장 (다른 도구와 연동 시)
df_parquet.write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/tmp/spark_tutorial/output/employees_output.csv")

print("CSV 저장 완료!")

CSV 저장 완료!


Part 5: DataFrame 확인 (Inspection)

In [15]:
# -----------------------------------------------------------------------------
# 스키마 확인: printSchema()
# -----------------------------------------------------------------------------

# printSchema(): 컬럼명, 데이터 타입, nullable 여부를 트리 형태로 출력
# - root: 최상위
# - |-- 컬럼명: 타입 (nullable = true/false)
print("=== 스키마 확인 ===")
df_csv.printSchema()

=== 스키마 확인 ===
root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_manager: boolean (nullable = true)



In [16]:
# -----------------------------------------------------------------------------
# 데이터 미리보기: show()
# -----------------------------------------------------------------------------

# show(): 기본 20행 출력
# show(n): n행 출력
# show(n, truncate=False): 컬럼 내용 잘리지 않게 출력
# show(n, truncate=10): 10자까지만 출력
# show(vertical=True): 세로로 출력 (컬럼 많을 때)

print("=== 기본 show() ===")
df_csv.show(5)  # 5행만 출력

print("=== truncate=False (잘림 없이) ===")
df_csv.show(3, truncate=False)

print("=== vertical=True (세로 출력) ===")
df_csv.show(2, vertical=True)

=== 기본 show() ===
+------+----------+----------+--------+----------+---+----------+
|emp_id|      name|department|  salary| hire_date|age|is_manager|
+------+----------+----------+--------+----------+---+----------+
|  E001|Employee_1|        HR| 92251.0|2020-01-01| 28|     false|
|  E002|Employee_2|   Finance| 62662.0|2020-01-08| 43|     false|
|  E003|Employee_3| Marketing| 48392.0|2020-01-15| 50|     false|
|  E004|Employee_4|   Finance| 70535.0|2020-01-22| 27|     false|
|  E005|Employee_5|   Finance|118603.0|2020-01-29| 43|     false|
+------+----------+----------+--------+----------+---+----------+
only showing top 5 rows

=== truncate=False (잘림 없이) ===
+------+----------+----------+-------+----------+---+----------+
|emp_id|name      |department|salary |hire_date |age|is_manager|
+------+----------+----------+-------+----------+---+----------+
|E001  |Employee_1|HR        |92251.0|2020-01-01|28 |false     |
|E002  |Employee_2|Finance   |62662.0|2020-01-08|43 |false     |
|E003  

In [17]:
# -----------------------------------------------------------------------------
# 행/컬럼 수 확인
# -----------------------------------------------------------------------------

# count(): 전체 행 수 반환 (Action이라 실행됨)
row_count = df_csv.count()
print(f"행 수: {row_count:,}")

# columns: 컬럼명 리스트 반환 (속성)
col_list = df_csv.columns
print(f"컬럼 목록: {col_list}")
print(f"컬럼 수: {len(col_list)}")

# dtypes: (컬럼명, 타입) 튜플 리스트 반환
print(f"컬럼별 타입: {df_csv.dtypes}")

행 수: 100
컬럼 목록: ['emp_id', 'name', 'department', 'salary', 'hire_date', 'age', 'is_manager']
컬럼 수: 7
컬럼별 타입: [('emp_id', 'string'), ('name', 'string'), ('department', 'string'), ('salary', 'double'), ('hire_date', 'date'), ('age', 'int'), ('is_manager', 'boolean')]


In [18]:
# -----------------------------------------------------------------------------
# 통계 요약: describe()
# -----------------------------------------------------------------------------

# describe(): 숫자형 컬럼의 기본 통계 (count, mean, stddev, min, max)
# 반환값이 DataFrame이므로 show() 호출 필요
print("=== 통계 요약 ===")
df_csv.describe().show()

# 특정 컬럼만 통계
print("=== salary 컬럼 통계 ===")
df_csv.describe("salary", "age").show()

=== 통계 요약 ===


26/01/19 07:40:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------+-----------+-----------+------------------+-----------------+
|summary|emp_id|       name| department|            salary|              age|
+-------+------+-----------+-----------+------------------+-----------------+
|  count|   100|        100|         96|                94|              100|
|   mean|  NULL|       NULL|       NULL| 80864.46808510639|            38.83|
| stddev|  NULL|       NULL|       NULL|22171.614668892573|9.084402216786948|
|    min|  E001| Employee_1|Engineering|           40206.0|               25|
|    max|  E100|Employee_99|      Sales|          119309.0|               54|
+-------+------+-----------+-----------+------------------+-----------------+

=== salary 컬럼 통계 ===
+-------+------------------+-----------------+
|summary|            salary|              age|
+-------+------------------+-----------------+
|  count|                94|              100|
|   mean| 80864.46808510639|            38.83|
| stddev|22171.614668892573|9.08440221678

In [20]:
# -----------------------------------------------------------------------------
# 유용한 추가 메서드들
# -----------------------------------------------------------------------------

# head(n): 처음 n행을 Row 객체 리스트로 반환
first_row = df_csv.head(1)
print(f"첫 번째 행: {first_row}")

# first(): 첫 번째 행을 Row 객체로 반환
first = df_csv.first()
print(f"first(): {first}")

# take(n): head(n)과 동일
top3 = df_csv.take(3)
print(f"take(3): {len(top3)}개 Row")

# collect(): 전체 데이터를 Driver로 가져옴 (주의: 대용량 시 OOM!)
# 작은 데이터에서만 사용
# all_rows = df_csv.collect()

# distinct(): 중복 제거된 행 수
dept_count = df_csv.select("department").distinct().count()
print(f"부서 종류: {dept_count}개")

첫 번째 행: [Row(emp_id='E001', name='Employee_1', department='HR', salary=92251.0, hire_date=datetime.date(2020, 1, 1), age=28, is_manager=False)]
first(): Row(emp_id='E001', name='Employee_1', department='HR', salary=92251.0, hire_date=datetime.date(2020, 1, 1), age=28, is_manager=False)
take(3): 3개 Row
부서 종류: 6개
